# tạo chunk

In [5]:
from underthesea import sent_tokenize
from langchain_text_splitters import CharacterTextSplitter
from typing import List
import json
import os
from tqdm import tqdm

class VietnameseTextProcessor:
    def __init__(self, max_length: int = 5000, overlap: int = 100):
        self.max_length = max_length
        self.overlap = overlap
        
    def read_text_file(self, file_path: str) -> str:
        """Đọc file văn bản với encoding utf-8"""
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def preprocess_text(self, text: str) -> str:
        """Tiền xử lý văn bản: giữ nguyên định dạng và dấu câu"""
        text = text.replace('\r\n', '\n')
        text = '\n'.join(line.strip() for line in text.split('\n'))
        return text
    
    def split_into_sentences(self, text: str) -> List[str]:
        """Tách văn bản thành các câu sử dụng Underthesea"""
        sentences = sent_tokenize(text)
        valid_sentences = [s.strip() for s in sentences if len(s.split()) > 5]
        return valid_sentences
    
    def create_chunks(self, sentences: List[str]) -> List[str]:
        """Ghép các câu thành chunk sử dụng LangChain"""
        splitter = CharacterTextSplitter(
            chunk_size=self.max_length,
            chunk_overlap=self.overlap,
            separator="\n",
            length_function=len
        )
        text = '\n'.join(sentences)
        chunks = splitter.split_text(text)
        return chunks

/Users/phucnt/.pyenv/versions/3.11.10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def main():
    processor = VietnameseTextProcessor(max_length=2000, overlap=200)
    
    input_folder = "data-raw/Thich Nhat Hanh"  # Thư mục chứa các file văn bản
    output_file = "chunks.json"  # File lưu chunks đầu ra

    all_chunks = []

    # Lặp qua tất cả các file trong thư mục
    files = [f for f in os.listdir(input_folder) if f.endswith('.txt')]
    
    for filename in files:
        file_path = os.path.join(input_folder, filename)
        
        # Đọc và tiền xử lý văn bản
        text = processor.read_text_file(file_path)
        text = processor.preprocess_text(text)
        
        # Tách câu và tạo chunks
        sentences = processor.split_into_sentences(text)
        chunks = processor.create_chunks(sentences)
        
        all_chunks.extend(chunks)  # Gộp vào danh sách tổng

    # Lưu tất cả chunks vào file JSON (tuỳ chọn)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)

    print(f"Số lượng chunks đã tạo: {len(all_chunks)}")
    print(f"Chunks được lưu tại: {output_file}")
    # Kiểm tra độ dài văn bản đầu vào
    print(f"Độ dài văn bản đầu vào: {len(text)}")

    # Kiểm tra độ dài từng chunk
    chunks = processor.create_chunks(sentences)
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i + 1} - Độ dài: {len(chunk)}")

if __name__ == "__main__":
    main()

Số lượng chunks đã tạo: 34
Chunks được lưu tại: chunks.json
Độ dài văn bản đầu vào: 60470
Chunk 1 - Độ dài: 1988
Chunk 2 - Độ dài: 1933
Chunk 3 - Độ dài: 1983
Chunk 4 - Độ dài: 1962
Chunk 5 - Độ dài: 1899
Chunk 6 - Độ dài: 1888
Chunk 7 - Độ dài: 1894
Chunk 8 - Độ dài: 1983
Chunk 9 - Độ dài: 1885
Chunk 10 - Độ dài: 1945
Chunk 11 - Độ dài: 1976
Chunk 12 - Độ dài: 1978
Chunk 13 - Độ dài: 1991
Chunk 14 - Độ dài: 1985
Chunk 15 - Độ dài: 1991
Chunk 16 - Độ dài: 1985
Chunk 17 - Độ dài: 1952
Chunk 18 - Độ dài: 1955
Chunk 19 - Độ dài: 1985
Chunk 20 - Độ dài: 1901
Chunk 21 - Độ dài: 1881
Chunk 22 - Độ dài: 1708
Chunk 23 - Độ dài: 1918
Chunk 24 - Độ dài: 1987
Chunk 25 - Độ dài: 1831
Chunk 26 - Độ dài: 1965
Chunk 27 - Độ dài: 1906
Chunk 28 - Độ dài: 1931
Chunk 29 - Độ dài: 1917
Chunk 30 - Độ dài: 1956
Chunk 31 - Độ dài: 1926
Chunk 32 - Độ dài: 1946
Chunk 33 - Độ dài: 1979
Chunk 34 - Độ dài: 263


# tạo sample

In [19]:
import os
import openai

# Thiết lập API key cho OpenAI từ biến môi trường
openai.api_key = os.getenv('OPENAI_API_KEY')

# Kiểm tra xem API key có được thiết lập hay không
if openai.api_key is None:
    raise ValueError("API key chưa được thiết lập. Vui lòng thiết lập biến môi trường 'OPENAI_API_KEY'.")

def test_openai_api(prompt: str):
    """Gọi API OpenAI với một prompt và in ra phản hồi."""
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Bạn có thể thay đổi mô hình nếu cần
            messages=[
                {"role": "system", "content": "Bạn là một trợ lý hữu ích, am hiểu về Phật Pháp."},
                {"role": "user", "content": f"""
                    Đây là đoạn trích từ sách của Thích Nhất Hạnh:
  "Khi tiếng chuông đầu báo hiệu giờ thiền tọa vọng lên, ta nên lập tức nhiếp tâm vào thiền quán, dù lúc đó ta đang ở đâu và đang làm gì.\nTa thong thả đứng dậy, đi rửa mặt hay rửa tay, mặc áo thiền tọa và khởi sự bước từng bước chân chậm rãi hướng về thiền đường, trong khi chuông vẫn tiếp tục vọng lên ba hồi ba tiếng.\nTrong suốt thời gian đó ta giữ chánh niệm, và ta có thể nương vào tiếng chuông và vào hơi thở để giữ chánh niệm.\nTa không đợi đến khi ngồi xuống trong thiền đường mới bắt đầu nhiếp niệm.\nTa nhiếp niệm ngay từ lúc nghe tiếng chuông đầu.\nVào thiền đường\nVào Thiền Đường\nThấy Chân Tâm\nMột Ngồi Xuống\nDứt Trầm Luân\nThiền đường là một chốn thanh tịnh, u nhã; một nơi mà ta có thể nhiếp niệm dễ dàng.\nVừa bước vào cửa thiền đường, ta nên dừng lại đưa mắt nhìn bên trong và đọc thầm bài kệ trên, trước khi ta chắp tay búp sen để xá Bụt.\nTâm quên lãng là vọng tâm, tâm tỉnh thức là chân tâm; khi ta ngồi xuống tư thế hoa sen và nhiếp tâm tỉnh thức thì mọi vọng tưởng điên đảo đều tan biến, không còn bủa vây và dẫn dắt ta đi vào cõi lãng quên nữa.\nPhiền não và vọng tưởng làm cho ta chìm đắm trong sinh tử, trong lãng quên.\nMột phen ngồi xuống là ta dứt được mọi trầm luân.\nTrầm luân có nghĩa là chìm đắm và trôi lăn trong biển quên lãng và phiền não.\nNgồi xuống\nNgồi Đây Ngồi Cội Bồ Đề\nVững Tâm Chính Niệm Không Hề Lãng Xao\nLoại cây Ficus Religiosa ấy sở dĩ được gọi là cây bồ đề bởi vì đức Bụt đã thực hiện được sự tỉnh thức toàn vẹn khi ngồi dưới gốc một cây ấy.\nBồ đề (bodhi) có nghĩa là tỉnh thức hay giác ngộ.\nNơi Phật thành đạo bây giờ được gọi là Bồ Đề Đạo Tràng (Bodhigayā).\nHiện có một tháp lớn được xây lên tại đó để kỷ niệm ngày thành đạo.\nCây bồ đề vẫn còn đó, sum suê và đẹp đẽ, tuy chỉ là con cháu của cây bồ đề năm xưa.\nTrong những năm gần đây, nhiều nước Phật giáo đã đến dựng chùa xung quanh Bồ Đề Đạo Tràng, mỗi nước một ngôi.\nĐứng trước tọa cụ của mình, bạn chắp tay xá rồi thầm niệm bài kệ này trước khi ngồi xuống."                     Đóng vai một tì kheo, hãy tạo ra 5 câu hỏi sâu sắc về đoạn văn trên.
                    Đảm bảo rằng các câu hỏi có thể khám phá các khía cạnh khác nhau và đầy đủ nhất của nội dung. 
                    Từ các câu hỏi, đóng vai Thích Nhất Hạnh, hãy tạo các câu trả tương ứng từ đoạn văn gốc.
                    Đảm bảo mô phỏng văn phong của đoạn văn gốc nhất có thể.
                    Câu trả lời dài và đầy đủ nhất có thể, nhưng không dài hơn đoạn văn gốc.
                    Hãy trả về kết quả dưới dạng JSON như sau:
                    {{
                        "questions": [
                            "Câu hỏi 1",
                            "Câu hỏi 2"
                        ],
                        answers: [
                            "Câu trả lời 1",
                            "Câu trả lời 2"
                        ]
                    }}
                """},
            ],
            temperature=0.5,  # Điều chỉnh độ sáng tạo của phản hồi
            response_format={"type": "json_object"}
        )
        
        # In ra nội dung phản hồi
        print("Phản hồi từ OpenAI:", response.choices[0].message.content)
    
    except Exception as e:
        print("Đã xảy ra lỗi khi gọi API:", e)

# Gọi hàm với một prompt mẫu
test_openai_api("")

Phản hồi từ OpenAI: {
    "questions": [
        "Tại sao việc nhiếp tâm ngay từ khi nghe tiếng chuông là quan trọng trong thiền quán?",
        "Ý nghĩa của việc chắp tay búp sen và xá Bụt khi vào thiền đường là gì?",
        "Cách mà tâm tỉnh thức giúp ta thoát khỏi phiền não và vọng tưởng được thể hiện như thế nào?",
        "Tại sao cây bồ đề lại có vai trò đặc biệt trong việc giác ngộ của Đức Phật?",
        "Thế nào là sự liên kết giữa việc ngồi thiền và việc dứt trầm luân trong sinh tử?"
    ],
    "answers": [
        "Việc nhiếp tâm ngay từ khi nghe tiếng chuông là quan trọng vì nó giúp ta giữ chánh niệm từ những giây phút đầu tiên, không để cho tâm trí lang thang vào những suy nghĩ khác. Khi ta nghe tiếng chuông, đó là dấu hiệu để ta trở về với hiện tại, và việc này tạo điều kiện cho ta dễ dàng vào thiền quán, giúp ta không bị lạc lối trong những vọng tưởng điên đảo.",
        "Chắp tay búp sen và xá Bụt khi vào thiền đường không chỉ là một hành động tôn kính, mà còn là một c